<a href="https://colab.research.google.com/github/LeftSpoonFish/Network-Graph-/blob/main/Network_Graph_from_Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
import networkx as nx
import spacy
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


# Sample text for analysis
text = '''
In a galaxy far, far away, amidst the swirling nebulae and distant star systems, lies the ancient planet of Alderaan. Here, beneath the watchful gaze of the towering mountains and lush forests, a young Jedi apprentice named Luke Skywalker embarks on an epic journey that will forever alter the course of history.
With his faithful droid companion, R2-D2, by his side, Luke sets out to uncover the truth about his mysterious past and unlock the secrets of the Force. Guided by the wisdom of the wise Jedi Master, Obi-Wan Kenobi, Luke learns the ways of the Force and hones his skills in preparation for the battles that lie ahead.
But the path to enlightenment is fraught with peril, and Luke soon finds himself entangled in a web of intrigue and deception. From the bustling streets of Coruscant to the desolate sands of Tatooine, he encounters a diverse array of beings, each with their own agendas and ambitions. Along the way, he forms unlikely alliances with smugglers, bounty hunters, and freedom fighters, all united in their quest to overthrow the tyrannical rule of the Galactic Empire.
As the conflict escalates, Luke discovers his true destiny as the last hope for the galaxy. With the fate of countless worlds hanging in the balance, he must confront his deepest fears and confront the dark forces that threaten to engulf the galaxy in chaos. With the power of the Force as his guide and the courage of his convictions as his strength, Luke embarks on a daring mission to restore peace and justice to the galaxy once and for all.
In the heart-pounding climax, Luke faces off against the sinister Sith Lord, Darth Vader, in a battle that will decide the fate of the galaxy. With lightsabers blazing and the fate of the universe hanging in the balance, Luke must draw upon all his training and courage to emerge victorious and fulfill his destiny as a Jedi Knight.
In the end, as the twin suns of Tatooine set on the horizon, Luke Skywalker stands triumphant, a beacon of hope for a galaxy in turmoil. With his friends by his side and the Force as his ally, he vows to continue his journey, exploring new worlds and seeking out new adventures in the endless expanse of the cosmos. For in the vastness of space, there are always new horizons to explore and new stories waiting to be told.
'''

# Function to create a network graph from text and save nodes and edges to CSV files
def create_network(text):
    entities = [word for word, pos in pos_tag(word_tokenize(text)) if pos == 'NNP']
    G = nx.Graph()
    for entity in set(entities):
        G.add_node(entity)
    for i in range(len(entities)):
        for j in range(i+1, len(entities)):
            if entities[i] != entities[j]:
                G.add_edge(entities[i], entities[j])

    return G

# Function to save nodes and edges DataFrames to a directory
def save_data_frames_to_directory(directory, nodes_df, edges_df):
    if not os.path.exists(directory):
        os.makedirs(directory)
    nodes_df.to_csv(os.path.join(directory, 'network_graph_nodes.csv'), index=False)
    edges_df.to_csv(os.path.join(directory, 'network_graph_edges.csv'), index=False)

# Create the network graph and get nodes and edges DataFrames
G = create_network(text)
nodes_df = pd.DataFrame({'node': list(G.nodes)})
edges_df = pd.DataFrame({'source': [e[0] for e in G.edges], 'target': [e[1] for e in G.edges]})

# Save nodes and edges DataFrames to a directory
save_data_frames_to_directory('network_data', nodes_df, edges_df)

# Load CSV files into DataFrames
nodes_df = pd.read_csv("network_data/network_graph_nodes.csv")
edges_df = pd.read_csv("network_data/network_graph_edges.csv")

# Display the first few rows of each DataFrame
print("Nodes DataFrame:")
print(nodes_df.head())

print("\nEdges DataFrame:")
print(edges_df.head())

# Define a title for the graph
title = "Network Graph"

# Create the network graph
G = nx.Graph()

# Add nodes from nodes_df
G.add_nodes_from(nodes_df['node'])

# Add edges from edges_df
G.add_edges_from(edges_df.values)

# Plot the 3D network graph with labels and hover information
pos = nx.spring_layout(G, dim=3)
edge_x = []
edge_y = []
edge_z = []
for edge in G.edges():
    source_pos = pos[edge[0]]
    target_pos = pos[edge[1]]
    edge_x += [source_pos[0], target_pos[0], None]
    edge_y += [source_pos[1], target_pos[1], None]
    edge_z += [source_pos[2], target_pos[2], None]

edge_trace = go.Scatter3d(
    x=edge_x, y=edge_y, z=edge_z,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

node_x = []
node_y = []
node_z = []
node_names = []  # List to store node names for labels
node_hoverinfo = []  # List to store hover information for nodes
for node in G.nodes():
    x, y, z = pos[node]
    node_x.append(x)
    node_y.append(y)
    node_z.append(z)
    node_names.append(node)  # Store node name for label
    connected_nodes = '<br>'.join(G.neighbors(node))  # Display connected nodes as a vertical list
    node_hoverinfo.append(f"<b>Node:</b> {node}<br><b>Connected Nodes:</b><br>{connected_nodes}")

node_trace = go.Scatter3d(
    x=node_x, y=node_y, z=node_z,
    mode='markers+text',  # Add text labels to markers
    marker=dict(symbol='circle',
                size=5,
                color='blue',
                line=dict(color='black', width=0.5)),
    text=node_names,  # Set node names as text labels
    hoverinfo='text',
    hovertext=node_hoverinfo)  # Set custom hover information

fig = go.Figure(data=[edge_trace, node_trace],
                layout=go.Layout(
                    title=title,
                    titlefont_size=16,
                    showlegend=False,
                    hovermode='closest',
                    scene=dict(
                        xaxis=dict(showticklabels=False, showbackground=False, title='', visible=False),
                        yaxis=dict(showticklabels=False, showbackground=False, title='', visible=False),
                        zaxis=dict(showticklabels=False, showbackground=False, title='', visible=False),
                        bgcolor='black'
                    )
                ))
fig.show()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Nodes DataFrame:
       node
0     Vader
1     R2-D2
2    Kenobi
3  Tatooine
4    Knight

Edges DataFrame:
  source     target
0  Vader   Alderaan
1  Vader       Jedi
2  Vader       Luke
3  Vader  Skywalker
4  Vader      R2-D2
